# import libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import anndata

from scopeseq.Bead_Intensity import BeadIntensity
from scopeseq.Cell_Intensity import CellIntensity
from scopeseq.Cell_Bead_Link import CellBeadLink
from scopeseq.Image_Seq_Link import ImageSeqLink
from scopeseq.Seq import CountMatrix
from scopeseq.Cluster import Cluster
from scopeseq.ScHPF import ScHPF

from scopeseq.utils import pickle_dump, pickle_load, merge_anndata, subsample_cells_anndata, filter_cells_anndata
from scopeseq.plot import plt_map
from scopeseq.stats import fit_gaussian, fit_double_gaussian
from scopeseq.clustering import cptt


"""
SCOPE-seq2 data processing
"""


# parameters

In [ ]:

project_folder = 'OUTDIR'
prefix = 'PREFIX'

lane = 0 
total_lanes = 5
cell_channels = ['GFP', 'TRITC', 'CY5'] # fluorescence cell imaging channels
bead_channels = ['bf', 'cy5', 'cy3'] # fluorescence bead imaging channels


# generate cell intensity

In [ ]:

cell_intensity = CellIntensity(project_folder)
cell_intensity.set_device(n_lane=lane, total_lanes=total_lanes, d_th=40)
cell_intensity.set_well_scan(well_channel=cell_channels)
cell_intensity.generate_well_intensity() # get average intensity of each microwell from ImageJ output

# get fluorescence threshold of live and/or dead cell stain to identify wells with cells
th_TRITC_well, fit_TRITC = fit_double_gaussian(cell_intensity._well['TRITC']['Mean'],
                            'Calcein', project_folder + 'analysis/well.calcein.pdf',
                            tail='upper', var=0.2)
th_CY5_well, fit_CY5 = fit_gaussian(cell_intensity._well['CY5']['Mean'],
                            'Annexin', project_folder + 'analysis/well.annexin.pdf',
                            tail='upper', var=0.2)
cell_intensity.cell_filter(filter_th={'TRITC': th_TRITC_well, 'CY5':th_CY5_well})

# write imaging coordinates of wells with cells into 'cell/sc_image.matrix.txt'
cell_intensity.generate_cell_intensity(cell_channel=cell_channels)

# write object
pickle_dump(cell_intensity, project_folder + 'cell_intensity.'+str(lane)+'.obj') 
# load object
cell_intensity = pickle_load(project_folder + 'cell_intensity.'+str(lane)+'.obj')



# optional: plot to check cell intensity

In [ ]:
image_matrix = pd.read_csv(project_folder + 'cell/sc_image.matrix.txt', sep='\t', index_col=0)

# get cell fluorescence threshold based on distribution fitness
th_TRITC_cell, fit_TRITC = fit_double_gaussian(image_matrix['TRITC_Mean_norm'],
                            'Calcein Normalized Mean', project_folder + 'analysis/cell.calcein.pdf',
                            tail='lower', var=0.2)
image_matrix_live = image_matrix[image_matrix['TRITC_Mean_norm']>th_TRITC_cell]
th_GFP_cell, fit_GFP = fit_double_gaussian(image_matrix_live['GFP_Mean_norm'],
                        'Caspase Normalized Mean', project_folder + 'analysis/cell.caspase.pdf',
                        tail='lower', var=0.2)
th_CY5_cell, fit_CY5 = fit_double_gaussian(image_matrix_live['CY5_Mean_norm'],
                            'Annexin Normalized Mean', project_folder + 'analysis/cell.annexin.pdf',
                            tail='lower', var=0.2)

pdf_outfile = project_folder + 'analysis/image.pdf'
with PdfPages(pdf_outfile) as pdf:
    plt.scatter(np.log2(image_matrix_live['GFP_Mean_norm']), np.log2(image_matrix_live['CY5_Mean_norm']), s=1)
    plt.xlabel('log2(Caspase Normalized Fluorescence)')
    plt.ylabel('log2(Annexin Normalized Fluorescence)')
    plt.title('Live cells')
    plt.axis('square')
    pdf.savefig(bbox_inches='tight')
    plt.close()


# bead optical demultiplexing

In [ ]:
bead_intensity = BeadIntensity(project_folder)
bead_intensity.set_device(n_lane=lane, total_lanes=total_lanes, patches_per_lane=6, first_border=200, border_gap=3350, d_th=25)
bead_intensity.set_demultiplexing(bead_channel=bead_channels, round_number=8,
                                  barcode_ref_fn = 'reference/192_8mer_seq_reference_2022.csv')
bead_intensity.generate_bead_intensity() # get average intensity of each microwell from ImageJ output
bead_intensity.probe_norm() # distribution normalization
# optical demultiplexing using core-by-core method
bead_intensity.obc_calling_core(no_signal_th=2**10) 
# optical demultiplexing using correlation method
bead_intensity.obc_calling_correlation(no_signal_th=2**10) 
# write object
pickle_dump(bead_intensity, project_folder + 'bead_intensity.'+str(lane)+'.obj')
# load object
bead_intensity = pickle_load(project_folder + 'bead_intensity.'+str(lane)+'.obj')


# link bead to cell

In [ ]:
cell_bead_link = CellBeadLink(project_folder)
cell_bead_link.set_bead_cell(bead_fn=project_folder + 'bead_intensity.'+str(lane)+'.obj',
                             cell_fn=project_folder + 'cell_intensity.'+str(lane)+'.obj',
                             landmark_fn=project_folder + 'bead/intensity_data/UL_BR_'+str(lane)+'.csv')
cell_bead_link.link_obc_cell()
# write object
pickle_dump(cell_bead_link, project_folder + 'cell_bead_link.'+str(lane)+'.obj')
# load object
cell_bead_link = pickle_load(project_folder + 'cell_bead_link.'+str(lane)+'.obj')


# link imaging to sequencing

In [ ]:
image_seq_link = ImageSeqLink(project_folder)
image_seq_link.set_image(image_fn=project_folder + 'cell_bead_link_.'+str(lane)+'.obj')
image_seq_link.set_sequencing(cell_id=project_folder + 'seq/'+prefix+'.edrops.matrix.hist.txt', use='index') # matrix.hist.txt file is from SCOPE-seq2 scRNA-seq fastq processing pipeline
image_seq_link.seq_image_link(replace={18: 14, 19: 15, 20: 17, 21: 18, 22: 19, 23: 20}) # map bead patch index to illumina sequencing index {bead: seq}
# write object
pickle_dump(image_seq_link, project_folder + 'image_seq_link.'+str(lane)+'.obj')
# load object
image_seq_link = pickle_load(project_folder + 'image_seq_link.'+str(lane)+'.obj')

# optional
# merge optial demultiplexing result from different methods 
image_seq_link.merge(obj1=project_folder + 'image_seq_link_correlation.'+str(lane)+'.obj', obj2=project_folder + 'image_seq_link_core.'+str(lane)+'.obj')

# generate linked single cell RNA-seq and imaging data
image_seq_link.generate_sc_measurements(seq_hist=project_folder + 'seq/'+prefix+'.edrops.matrix.hist.txt',
                                        seq_matrix=project_folder + 'seq/'+prefix+'.edrops.matrix.txt')
# write object
pickle_dump(image_seq_link, project_folder + 'image_seq_link.'+str(lane)+'.obj')


# cell hashing for sample demultiplexing

In [ ]:
from scopeseq.HTO import hto

# hashtag qualtity check, and write sample name mapped table to project_folder+prefix+'_HTO.demultiplexed.txt'
htos = ['Hashtag1','Hashtag2','Hashtag3']
samples = ['DMSO','Etoposide','Panobinostat']
umap_file = project_folder+'seq/'+prefix+'.markers.dcorrSP.umap.txt'
hashtag = hto(project_folder,prefix,htos,samples, umap_file) 


# sequencing

In [ ]:
sample_name = 'SAMPLE'
batch_name = 'BATCH'

count_matrix = CountMatrix(project_folder)
count_matrix.set_sequencing(matrix_file=project_folder + 'seq/'+prefix+'.edrops.matrix.txt',
                            hist_file=project_folder + 'seq/'+prefix+'.edrops.matrix.hist.txt', # These files are from SCOPE-seq2 scRNA-seq fastq processing pipeline
                            sample_index={sample_name:[14,15,17,18,19,20,21,22,23,25,27,28]},
                            batch_index={batch_name: [14,15,17,18,19,20,21,22,23,25,27,28]}) # sequencing index

# optional: if use cell hashing
count_matrix._cell_summary['sample']=hashtag['sample'].values

count_matrix.seq_summary()
# generate anndata data storage. write loom file
count_matrix_anndata = count_matrix.to_anndata()
# read in loom file
count_matrix_anndata = anndata.read_loom(project_folder + 'count_matrix.loom', obs_names='', var_names='')



# embedding, clustering

In [ ]:
# using diffexcluster pipeline
cluster = Cluster(project_folder)
cluster.set_cluster(mtx=project_folder + 'count_matrix.loom',
                    pg=project_folder + 'seq/'+prefix+'.markers.dcorrSP.pg.txt',
                    umap=project_folder + 'seq/'+prefix+'.markers.dcorrSP.umap.txt') # These files are from diffexcluster processing pipeline
# link image
for i in samples+['Doublet']:
    cluster.link_image(image_file=project_folder + 'analysis/sc_image.matrix.linked.txt', sample=i, batch=batch_name)


# image

# get the fluorescence threshold from vehicle control
neg_control = cluster._anndata[(cluster._anndata.obs['sample']=='DMSO')]
th_calcein, fit_calcein = fit_gaussian(neg_control['TRITC_Mean_norm'],
                            'calcein', project_folder + 'analysis/neg.calcein.pdf',
                            tail='lower', var=0.2)
th_caspase, fit_caspase = fit_gaussian(neg_control['GFP_Mean_norm'],
                            'caspase', project_folder + 'analysis/neg.caspase.pdf',
                            tail='upper', var=0.2)
th_annexin, fit_annexin = fit_gaussian(neg_control['CY5_Mean_norm'],
                            'annexin', project_folder + 'analysis/neg.annexin.pdf',
                            tail='upper', var=0.2)

# assign cell status
cluster._anndata.obs['cell_status'] = None
cluster._anndata.obs['cell_status'][((cluster._image['TRITC_Mean_norm']<th_calcein)).values] = 'dead'
cluster._anndata.obs['cell_status'][((cluster._image['TRITC_Mean_norm']>th_calcein)).values] = 'viable'
cluster._anndata.obs['cell_status'][((cluster._image['GFP_Mean_norm']>th_caspase) &
                                    (cluster._image['CY5_Mean_norm'] < th_annexin) &
                                    (cluster._image['TRITC_Mean_norm']>th_calcein)).values]= 'apoptosis_1'
cluster._anndata.obs['cell_status'][((cluster._image['GFP_Mean_norm']>th_caspase) &
                                    (cluster._image['CY5_Mean_norm'] > th_annexin) &
                                    (cluster._image['TRITC_Mean_norm']>th_calcein)).values] = 'apoptosis_2'
cluster._anndata.obs['cell_status'][((cluster._image['GFP_Mean_norm']<th_caspase) &
                                    (cluster._image['CY5_Mean_norm'] > th_annexin) &
                                    (cluster._image['TRITC_Mean_norm']>th_calcein)).values] = 'apoptosis_3'
cluster._anndata.obs[cell_channels] = cluster._image[[x+'_Mean_norm' for x in cell_channels]].values

cluster.to_anndata() # write obs information to annotation.txt, and anndata object to count_matrix.loom
# write object
pickle_dump(cluster, project_folder + 'cluster.obj')

# plot umap embedding colored by imaging phenotypes
pdf_outfile = project_folder + 'analysis/umap_image.pdf'
with PdfPages(pdf_outfile) as pdf:
    for i in cell_channels:
        plt.scatter(cluster._anndata.obsm['umap_emb'][:,0], cluster._anndata.obsm['umap_emb'][:,1],
                    c=np.log2(data[i+'_Mean_norm'].astype('float')), s=3, cmap='coolwarm')
        plt.colorbar()
        plt.xlabel('UMAP 1')
        plt.ylabel('UMAP 2')
        plt.title('colored by log2('+i+' normalized intensity)')
        pdf.savefig(bbox_inches='tight')
        plt.close()

    plt_map(pdf, x=cluster._anndata.obsm['umap_emb'], name='colored by cell status',
                color=mpl.cm.tab10.colors, group=cluster._anndata.obs['cell_status'], order=['live','apoptosis_1','apoptosis_2','apoptosis_3','dead'],
                t='category', s=5)


"""
Combind analysis of multiple SOCPE-seq2 experiments
And some example analysis functions: malignant cell identificaiton, cell annotation. 
"""


# project

In [ ]:
project_folder = 'OUTDIR'
datasets = pd.read_csv(project_folder+'datasets.txt', sep='\t',dtype='str')

# parameters
batch_order = ['0520','0608']
sample_order = ['DMSO', 'Etoposide', 'Panobinostat', 'Topotecan', 'Givinostat']
batch_sample_order = ['20220520_'+s for s in ['DMSO','Etoposide','Panobinostat']] + ['20220608_'+s for s in ['DMSO','Topotecan','Givinostat']]
status_order = ['viable','apoptosis_1','apoptosis_2','apoptosis_3']
cluster_color = mpl.cm.tab10.colors
batch_color =  mpl.cm.tab10.colors[8:10]
sample_color = mpl.cm.Dark2.colors[0:5]
status_color = mpl.cm.Set2.colors[7:8]+mpl.cm.Paired.colors[7:8]+mpl.cm.Paired.colors[9:10]+mpl.cm.Paired.colors[11:12]



# combine count matrix and cell annotations

In [ ]:
#subsample vehicle control cells
combined = merge_anndata(dirs=datasets['Path'],output_dir=project_folder)
combined_DMSO = subsample_cells_anndata(combined[combined.obs['sample']=='DMSO'], output_dir=project_folder)

# scHPF on subsampled vehicle control cells to find variable genes for downstream analysis
# please reference to the scHPF method
schpf = ScHPF(project_folder)
schpf.set_schpf(mtx=project_folder + 'schpf/filtered.mtx', genes=project_folder + 'schpf/genes.txt',
                schpf_joblib= project_folder + 'schpf/'+prefix+'.scHPF_K10_b0_5trials.joblib')
schpf.set_cell_annotation(annotation=project_folder + 'annotation_sub.txt')
# identify batch-specific/HSP/Ribosomal factors, and remove them from marker gene lists
schpf.plt_cell_score()
schpf.set_batch_factor(batch_factor=[0,2,9])
# identify top 100 genes of each key factors as variable marker genes
schpf.marker_genes()

# filter dataset based on marker genes expression and remove doublets
combined_single = filter_cells_anndata(combined, marker_file=project_folder+'schpf/marker_genes.txt')
combined_single.write_loom(project_folder + 'count_matrix.all_single.loom')
# write normalized anndata
from scipy import sparse
matrix_norm = cptt(combined_single.X.todense().T)
adata_norm = anndata.AnnData(sparse.csr_matrix(matrix_norm.T), obs = combined_single.obs, var=combined_single.var)
adata_norm.write_loom(project_folder + 'count_matrix.all_single.norm.loom')

# run clusterdiffex on filtered dataset.
combined_single.obsm['umap_emb'] = pd.read_csv(project_folder+'seq/'+prefix+'.fmarkers.dcorrSP.umap.txt', sep='\t',header=None).values
combined_single.obs['pgs'] = pd.read_csv(project_folder+'seq/'+prefix+'.fmarkers.dcorrSP.pg.txt', sep='\t',header=None).values
# optional merge cell clusters
pgs = [0,1,2,3,4,5,6,7,8,9,10,11]
pgs_after = [2,0,1,0,0,0,0,0,0,0,4,3]
combined_single.obs['pgs'] = map(combined_single.obs['pgs_all'],pgs,pgs_after)

# basic plot
pdf_outfile = project_folder + 'all.umap.pdf'
with PdfPages(pdf_outfile) as pdf:
    plt_map(pdf, x=combined_single.obsm['umap_emb'], name='batch',
            color=batch_color, group=combined_single.obs['batch'],
            order=batch_order,
            t='category', s=3)
    plt_map(pdf, x=combined_single.obsm['umap_emb'], name='sample',
            color=sample_color, group=combined_single.obs['sample'],
            order=sample_order,
            t='category', s=3)
    for i in range(len(sample_order)):
        x = combined_single[combined_single.obs['sample']==sample_order[i]].obsm['umap_emb']
        plt.scatter(x[:,0],x[:,1],color=sample_color[i], s=3)
        plt.colorbar()
        plt.title=sample_order[i]
        plt.xlabel('')
        plt.ylabel('')
        pdf.savefig(bbox_inches='tight')
        plt.close()
    plt_map(pdf, x=combined_single.obsm['umap_emb'], name='pgs',
            color=cluster_color, group=combined_single.obs['pgs'],
            t='category', s=3)
    for i in range(len(sample_order)):
        x = combined_single[combined_single.obs['sample']==sample_order[i]]
        plt_map(pdf, x=x.obsm['umap_emb'], name='pgs_'+sample_order[i],
            color=cluster_color, group=x.obs['pgs'],
            t='category', s=3)


# examine lineage marker gene expression and signature score of cell clusters

In [ ]:
from scopeseq.utils import load_genesets_fromtxt
from scopeseq.clustering import neftel_subtype

adata_norm = anndata.read_loom(project_folder + 'count_matrix.all_single.norm.loom', obs_names='', var_names='')
adata_norm.obs = combined_single.obs
adata_norm = adata_norm[adata_norm.obs['sample']=='DMSO']
matrix_norm = adata_norm.X.todense().T

# marker gene expression
from scipy.stats import zscore
gene = ['OLIG1','OLIG2','DLL3','DCX','APOE','CLU','SPARC','CD44','ANXA2','PCNA','RRM2','CCNE2','CCNB1','CDC20','AURKA']
matrix_norm_marker = pd.DataFrame()
for i in gene:
    exp_all = matrix_norm[(adata_norm.var['Gene']==i).values,:].tolist()[0]
    matrix_norm_marker[i] = exp_all
matrix_norm_marker_zscore = pd.DataFrame(zscore(matrix_norm_marker, axis=0),index=matrix_norm_marker.index, columns=matrix_norm_marker.columns)
matrix_norm_marker['pgs'] = adata_norm.obs['pgs_all'].values
matrix_norm_marker['batch'] = adata_norm.obs['batch'].values
matrix_norm_marker_zscore['pgs'] = adata_norm.obs['pgs_all'].values
matrix_norm_marker_zscore['batch'] = adata_norm.obs['batch'].values
matrix_norm_marker_mean = matrix_norm_marker.groupby(['pgs','batch']).mean()
matrix_norm_marker_mean_zscore = matrix_norm_marker_zscore.groupby(['pgs','batch']).mean()

pdf_outfile = project_folder + 'gene.pgs.heatmap.pdf'
with PdfPages(pdf_outfile) as pdf:
    p_color = [cluster_color[x] for x in [0,0,1,1,2,2,3,3,4,4]]
    b_color = [batch_color[x] for x in [0,1,0,1,0,1,0,1,0,1]]
    color = pd.DataFrame({'pgs':p_color, 'batch':b_color},index=matrix_norm_marker_mean.index)
    sns.clustermap(matrix_norm_marker_mean.T, cmap='Reds',cbar_kws={'label':'Averaged normalized Expression'},
                   row_cluster=False, col_cluster=False,
                   col_colors=color, vmin=0, vmax=3)
    plt.xlabel('')
    plt.ylabel('')
    pdf.savefig(bbox_inches='tight')
    plt.close()

    p_color = [cluster_color[x] for x in [0,0,1,1,2,2,3,3,4,4]]
    b_color = [batch_color[x] for x in [0,1,0,1,0,1,0,1,0,1]]
    color = pd.DataFrame({'pgs':p_color, 'batch':b_color}, index=matrix_norm_marker_mean_zscore.index)
    sns.clustermap(pd.DataFrame(matrix_norm_marker_mean_zscore.T), cmap='coolwarm',cbar_kws={'label':'zscore(Averaged normalized Expression)'},
                   row_cluster=False, col_cluster=False,
                   col_colors=color, vmin=-0.4, vmax=0.4)
    plt.xlabel('')
    plt.ylabel('')
    pdf.savefig(bbox_inches='tight')
    plt.close()


# gene signatures from Neftel et. al. 2019
genesets_infile = 'reference/tumor_hetero_module_genesets.txt'
genesets_dict = load_genesets_fromtxt(genesets_infile)

group='pgs'
b=100
hetero_2D, SC_all = neftel_subtype(adata_norm, genesets_infile, group=group, b=b)
hetero_2D.to_csv(project_folder+'ref.2D.'+group+'.'+str(b)+'.txt',sep='\t',header=False,index=False)
SC_all.to_csv(project_folder+'ref.SCall.'+group+'.'+str(b)+'.txt',sep='\t',index=False)

adata_norm.obsm['hetero_2D'] = hetero_2D.values
adata_norm.obsm['SC_all'] = SC_all.values
adata_norm.obs[SC_all.columns] = SC_all.values
adata_norm.obs['MES'] = adata_norm.obs[['MES1','MES2']].max(axis=1)
adata_norm.obs['NPC'] = adata_norm.obs[['NPC1','NPC2']].max(axis=1)

pdf_outfile = project_folder + 'signature.pgs.heatmap.pdf'
with PdfPages(pdf_outfile) as pdf:
    matrix_marker_mean = adata_norm.obs[['MES','AC','OPC','NPC','G1/S','G2/M','pgs','batch']].groupby(['pgs','batch']).mean()
    p_color = [cluster_color[x] for x in [0,0,1,1,2,2,3,3,4,4]]
    b_color = [batch_color[x] for x in [0,1,0,1,0,1,0,1,0,1]]
    color = pd.DataFrame({'pgs':p_color, 'batch':b_color},index=matrix_marker_mean.index)
    sns.clustermap(matrix_marker_mean.T, cmap='coolwarm',cbar_kws={'label':'Average meta-module score'},
                   row_cluster=False, col_cluster=False,
                   col_colors=color,
                   vmin=-0.4, vmax=0.4)
    plt.xlabel('')
    plt.ylabel('')
    pdf.savefig(bbox_inches='tight')
    plt.close()



# combine image

In [ ]:
# batch normalization
# calculate mean of negative control cells (DMSO & live) of each sample
cell_channels = ['GFP','TRITC','CY5']
neg_mean = combined_single.obs[['batch','sample','cell_status']+channels].groupby(['batch','sample','cell_status']).mean()
neg_mean = neg_mean.xs('DMSO', level='sample').xs('live', level='cell_status')
# normalize ctrl mean to the lowest in each fluorescence channel
neg_mean_ratio = neg_mean/neg_mean.min()
combined_single.obs[[s + '_batchnorm' for s in channels]] = \
    combined_single.obs.apply(lambda x: x[channels]/neg_mean_ratio.loc[x['batch']], axis=1)

# get threshold
neg_control = combined_single[(combined_single.obs['sample']=='DMSO')&(combined_single.obs['cell_status']!='dead')]
th_caspase, fit_caspase = fit_gaussian(neg_control.obs['GFP_batchnorm'],
                            'Caspase 3/7 Normalized Mean', project_folder + 'image/neg.caspase.pdf',
                            tail='upper', var=0.2)
th_annexin, fit_annexin = fit_gaussian(neg_control.obs['CY5_batchnorm'],
                            'Annexin V Normalized Mean', project_folder + 'image/neg.annexin.pdf',
                            tail='upper', var=0.2)

# assign cell status
combined_single.obs['cell_status_merge'] = None
combined_single.obs['cell_status_merge'][((combined_single.obs['cell_status']=='dead')).values] = 'dead'
combined_single.obs['cell_status_merge'][((combined_single.obs['cell_status']!='dead') &
                                    (combined_single.obs['cell_status']!='None')).values] = 'viable'
combined_single.obs['cell_status_merge'][((combined_single.obs['GFP_batchnorm']>th_caspase) &
                                    (combined_single.obs['CY5_batchnorm'] < th_annexin) &
                                    (combined_single.obs['cell_status']!='dead')).values]= 'apoptosis_1'
combined_single.obs['cell_status_merge'][((combined_single.obs['GFP_batchnorm']>th_caspase) &
                                    (combined_single.obs['CY5_batchnorm'] > th_annexin) &
                                    (combined_single.obs['cell_status']!='dead')).values] = 'apoptosis_2'
combined_single.obs['cell_status_merge'][((combined_single.obs['GFP_batchnorm']<th_caspase) &
                                    (combined_single.obs['CY5_batchnorm'] > th_annexin) &
                                    (combined_single.obs['cell_status']!='dead')).values] = 'apoptosis_3'

combined_single.obs.to_csv(project_folder+'annotation.image.txt',sep='\t',index=False)


# plot batch normalized imaging phenotypes

In [ ]:
combined_image_filter = combined_single[(combined_single.obs['cell_status']!='None')&(combined_single.obs['cell_status']!='dead')]

pdf_outfile = project_folder + 'image/image.pdf'
with PdfPages(pdf_outfile) as pdf:
    sns.scatterplot(x=np.log2(combined_image_filter.obs['GFP_batchnorm']), y=np.log2(combined_image_filter.obs['CY5_batchnorm']),
                    hue=combined_image_filter.obs['sample'], hue_order=sample_order, palette=sample_color,
                    s=3, linewidth=0)
    plt.xlabel('log2(Caspase 3/7 Normalized Mean)')
    plt.ylabel('log2(Annexin V Normalized Mean)')
    plt.axis('square')
    pdf.savefig(bbox_inches='tight')
    plt.close()

    sns.scatterplot(x=np.log2(combined_image_filter.obs['GFP_batchnorm']), y=np.log2(combined_image_filter.obs['CY5_batchnorm']),
                    hue=combined_image_filter.obs['cell_status_merge'], hue_order=status_order, palette=status_color,
                    s=3, linewidth=0)
    plt.xlabel('log2(Caspase 3/7 Normalized Mean)')
    plt.ylabel('log2(Annexin V Normalized Mean)')
    plt.axis('square')
    pdf.savefig(bbox_inches='tight')
    plt.close()

pdf_outfile = project_folder + 'image/cell_status.pdf'
with PdfPages(pdf_outfile) as pdf:
    plt_map(pdf, x=combined_image_filter.obsm['umap_emb'], name='sample',
            color=sample_color, group=combined_image_filter.obs['sample'].values, order=sample_order,
            t='category', s=5)
    plt_map(pdf, x=combined_image_filter.obsm['umap_emb'], name='pgs',
            color=cluster_color, group=combined_image_filter.obs['pgs'].values,
            t='category', s=5)
    plt_map(pdf, x=combined_image_filter.obsm['umap_emb'], name='cell_status_merge',
            color=status_color, group=combined_image_filter.obs['cell_status_merge'].values, order=status_order,
            t='category', s=5)
    for i in range(len(sample_order)):
        x = combined_image_filter[combined_image_filter.obs['sample']==sample_order[i]]
        plt_map(pdf, x=x.obsm['umap_emb'], name='cell_status_merge_'+sample_order[i],
            color=status_color, group=x.obs['cell_status_merge'].values, order=status_order,
            t='category', s=5)

    plt_map(pdf, x=combined_image_filter.obsm['umap_emb'], name='Calcein',
                color=np.log2(combined_image_filter.obs['TRITC_batchnorm']),
                cmap='Reds', t='float', s=5)
    plt_map(pdf, x=combined_image_filter.obsm['umap_emb'], name='Caspase 3/7',
                color=np.log2(combined_image_filter.obs['GFP_batchnorm']),
                cmap='cool', t='float', vmin=6, vmax=9, s=5)
    plt_map(pdf, x=combined_image_filter.obsm['umap_emb'], name='Annexin V',
                color=np.log2(combined_image_filter.obs['CY5_batchnorm']),
                cmap='cool', t='float', vmin=8, vmax=11, s=5)

    data = pd.crosstab(index=[combined_image_filter.obs['sample'],combined_image_filter.obs['pgs']],
                        columns=combined_image_filter.obs['cell_status_merge'], normalize='index')
    data = data[status_order]
    data = data.loc[sample_order]
    data.plot(kind='bar', stacked=True,color=status_color)
    plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
    plt.xlabel("Sample_cluster")
    plt.ylabel("Proportion")
    pdf.savefig(bbox_inches='tight')
    plt.close()
    

